In [ ]:
!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 72.1 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

from itertools import product
from math import sqrt

In [ ]:
m = gp.Model("Car Charging")

Restricted license - for non-production use only - expires 2024-10-28


In [ ]:
def compute_distance(loc1, loc2):

    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [ ]:
def solve_electric_car_chargers(c_coordinates, demand):
    neighborhood = [*range(1,21)]
    chargers = [*range(1,51)]
    existing, e_coordinates, e_capacity  = gp.multidict({
        1: [(1.3, 2.2), 5],
        2: [(12.5, 1.2), 30],
        3: [(5.33, 11), 20],
        4: [(6.56, 3.55), 30],
        5: [(1.02, 5.12), 15],
        6: [(3.12, 12.23), 30],
        7: [(5.5, 14.45), 30],
        8: [(6.15, 15.5), 5],
        9: [(16, 8.35), 20],
        10: [(13.25, 14.25), 5],
        11: [(8.51, 6.6), 15],
        12: [(1.25, 8.23), 30],
        13: [(0.23, 0.297897), 30],
        14: [(8.564, 14.956), 15],
        15: [(1.591, 1.89), 25],
        16: [(3.892, 1.5845), 15],
        17: [(5.525, 12.335), 25],
        18: [(11.2315, 3.6735), 5],
        19: [(3.3548, 12.543), 30],
        20: [(4.4565, 4.546), 5],
        21: [(1.26575, 6.43), 25],
        22: [(3.134, 6.675), 20],
        23: [(5.55, 6.155), 5],
        24: [(1.344, 2.352), 15],
        25: [(12.4675, 1.241), 30],
        26: [(5.1546, 0.5451), 15],
        27: [(6.557, 3.2455), 5],
        28: [(1.098, 5.0987), 5],
        29: [(3.087, 4.01576), 5],
        30: [(5.021467, 14.009), 5]

    })

    temporary, t_coordinates, t_capacity  = gp.multidict({
        31: [(12.365, 0.31), 5],
        32: [(13.56, 13.25), 5],
        33: [(5.51, 12.15), 10],
        34: [(1.52, 3.53), 5],
        35: [(3.52, 2.53), 10],
        36: [(8.75, 3.45), 5],
        37: [(31.5, 6.45), 10],
        38: [(33.5, 16.5), 5],
        39: [(15.5, 6.35), 10],
        40: [(11.5, 4.1), 10],
        41: [(3.45, 1.75), 10],
        42: [(4.15, 2.35), 5],
        43: [(13.5, 23.5), 10],
        44: [(13.59, 2.15), 5],
        45: [(14.53, 14.35), 5],
        46: [(1.35, 6.75), 5],
        47: [(3.53, 13.75), 10],
        48: [(15.56, 12.35), 5],
        49: [(3.534, 1.513), 10],
        50: [(5.345, 2.563), 10],
        71: [(14.5, 3.35), 5]
    })

    dcost = 0.05
    tfcost = 3000000

    f_coordinates = {}
    for e in existing:
        f_coordinates[e] = e_coordinates[e]

    for t in temporary:
        f_coordinates[t] = t_coordinates[t]

    cf = []

    for c in neighborhood:
        for f in chargers:
            tp = c,f
            cf.append(tp)

    distance = {(c,f): compute_distance(c_coordinates[c], f_coordinates[f]) for c, f in cf}

    y = m.addVars(temporary, vtype=GRB.BINARY, name='temporary')

    x = m.addVars(cf, vtype=GRB.CONTINUOUS, name='Assign')

    z = m.addVars(temporary, vtype=GRB.CONTINUOUS, name='addCap' )
    bigM = 1e9

    m.setObjective(gp.quicksum(dcost*distance[c,f]*x[c,f] for c,f in cf)
                   + tfcost*y.sum()
                   + bigM*z.sum(), GRB.MINIMIZE)

    demandConstrs = m.addConstrs((gp.quicksum(x[c,f] for f in chargers) == demand[c] for c in neighborhood), name='demandConstrs')

    existingCapConstrs = m.addConstrs((gp.quicksum(x[c,e]  for c in neighborhood) <= e_capacity[e] for e in existing ),
                                      name='existingCapConstrs')

    temporaryCapConstrs = m.addConstrs((gp.quicksum(x[c,t]  for c in neighborhood) -z[t] <= t_capacity[t]*y[t] for t in temporary ), name='temporaryCapConstrs')

    m.write('charge.lp')

    m.optimize()

    new_charger_cost = 0

    print(f"\n\n_____________Optimal costs______________________")
    for t in temporary:
        if (y[t].x > 0.5):
            new_charger_cost += tfcost*round(y[t].x)

    patient_allocation_cost = 0
    for c,f in cf:
        if x[c,f].x > 1e-6:
            patient_allocation_cost += dcost*round(distance[c,f]*x[c,f].x)

    print(f"The total cost of building COVID-19 temporary healhtcare facilities is ${new_charger_cost:,}")
    print(f"The total cost of allocating COVID-19 patients to healtcare facilities is ${patient_allocation_cost:,}")


    print(f"\n_____________Plan for temporary facilities______________________")
    for t in temporary:
        if (y[t].x > 0.5):
            print(f"Build a temporary facility at location {t}")

    print(f"\n_____________Plan to increase Capacity at temporary Facilities______________________")
    for t in temporary:
        if (z[t].x > 1e-6):
            print(f"Increase  temporary facility capacity at location {t} by {round(z[t].x)} beds")

    f_demand = {}

    print(f"\n_____________Allocation of county patients to COVID-19 healthcare facility______________________")
    for f in chargers:
        temp = 0
        for c in counties:
            allocation = round(x[c,f].x)
            if allocation > 0:
                print(f"{allocation} COVID-19 patients from county {c} are treated at facility {f} ")
            temp += allocation
        f_demand[f] = temp
        print(f"{temp} is the total number of COVID-19 patients that are treated at facility {f}. ")
        print(f"\n________________________________________________________________________________")

    total_demand = 0

    for c in counties:
        total_demand += demand[c]

    demand_satisfied = 0
    for f in chargers:
        demand_satisfied += f_demand[f]

    print(f"\n_____________Test demand = supply______________________")
    print(f"Total demand is: {total_demand:,} patients")
    print(f"Total demand satisfied is: {demand_satisfied:,} beds")

In [ ]:
neighborhood, coordinates, num_electric_cars  = gp.multidict({
    1: [(1, 1.5), 1179],
    2: [(3, 1), 133],
    3: [(5.5, 1.5), 23],
    4: [(1, 4.5 ), 339],
    5: [(3, 3.5), 303],
    6: [(5.5, 4.5), 527],
    7: [(1, 8), 469],
    8: [(3, 6), 234],
    9: [(4.5, 8), 4352],
    10: [(3.23, 1.25), 1235],
    11: [(7.5, 2.65), 1425],
    12: [(10, 4.85 ), 271],
    13: [(13, 3.15), 343],
    14: [(9.56, 4.85), 275],
    15: [(2.31, 18), 862],
    16: [(3.73, 2.36),114],
    17: [(14.5, 8.18), 455],
    18: [(4.75, 8.12), 326],
    19: [(1.16, 11.5), 735],
    20: [(0.3, 2.11), 132]

})


solve_electric_car_chargers(coordinates, num_electric_cars)

KeyError: ignored


neighborhood, coordinates, num_electric_cars  = gp.multidict({
    'San Francisco': [(1, 1.5), 11729],
    'Mountain View': [(3, 1), 1331],
    'Cupertino': [(5.5, 1.5), 232],
    'Menlo Park': [(1, 4.5 ), 3339],
    'San Ramon': [(3, 3.5), 3603],
    'Milpitas': [(5.5, 4.5), 527],
    'Oakland': [(1, 8), 4697],
    'Los Altos': [(3, 6), 2234],
    'Walnut Creek': [(4.5, 8), 4352],  
    'San Jose': [(2, 2.5), 2334],
    'Saratoga': [(3.23, 1.25), 7235],
    'Pleasanton': [(7.5, 2.65), 3425],
    'Pittsburgh': [(10, 4.85 ), 5771],
    'Fremont': [(13, 3.15), 2343],
    'Hayward': [(9.56, 4.85), 2725],
    'Daly City': [(2.31, 18), 8632],
    'Redwood City': [(3.73, 2.36),1145],
    'Foster City': [(14.5, 8.18), 4535],
    'Berkeley': [(4.75, 8.12), 3256],  
    'Santa Clara': [(1.16, 11.5), 7385],
    'Burlingame': [(0.3, 2.11), 1328],
    'Gilroy': [(7.53, 12.5), 2413],
    'Millbrae': [(11.34, 54.75 ), 865],
    'Mill Valley': [(5.63, 13.5), 2434],
    'Vallejo': [(15.5, 4.59), 2139],
    'Union City': [(11.5, 12.38), 1528],
    'Petaluma': [(13.45, 13.13), 2380],
    'Morgan Hill': [(4.51, 18.67), 1256],
    'Dublin': [(2.5, 1), 1835]

})